In [5]:
import pyspark
from pyspark.sql import SparkSession

In [6]:
spark = SparkSession.builder \
    .master("local[*]") \
    .appName("test") \
    .getOrCreate()

In [8]:
df_green = spark.read \
    .option("header", "true") \
    .csv('data/raw/green/2021/01/')

In [9]:
import pandas as pd

In [10]:
df_green_pd = pd.read_csv('data/raw/green/2021/01/green_tripdata_2021_01.csv.gz', nrows=1000)

In [11]:
spark.createDataFrame(df_green_pd).schema

StructType([StructField('VendorID', LongType(), True), StructField('lpep_pickup_datetime', StringType(), True), StructField('lpep_dropoff_datetime', StringType(), True), StructField('store_and_fwd_flag', StringType(), True), StructField('RatecodeID', LongType(), True), StructField('PULocationID', LongType(), True), StructField('DOLocationID', LongType(), True), StructField('passenger_count', LongType(), True), StructField('trip_distance', DoubleType(), True), StructField('fare_amount', DoubleType(), True), StructField('extra', DoubleType(), True), StructField('mta_tax', DoubleType(), True), StructField('tip_amount', DoubleType(), True), StructField('tolls_amount', DoubleType(), True), StructField('ehail_fee', DoubleType(), True), StructField('improvement_surcharge', DoubleType(), True), StructField('total_amount', DoubleType(), True), StructField('payment_type', LongType(), True), StructField('trip_type', LongType(), True), StructField('congestion_surcharge', DoubleType(), True)])

In [12]:
df_green.printSchema()

root
 |-- VendorID: string (nullable = true)
 |-- lpep_pickup_datetime: string (nullable = true)
 |-- lpep_dropoff_datetime: string (nullable = true)
 |-- store_and_fwd_flag: string (nullable = true)
 |-- RatecodeID: string (nullable = true)
 |-- PULocationID: string (nullable = true)
 |-- DOLocationID: string (nullable = true)
 |-- passenger_count: string (nullable = true)
 |-- trip_distance: string (nullable = true)
 |-- fare_amount: string (nullable = true)
 |-- extra: string (nullable = true)
 |-- mta_tax: string (nullable = true)
 |-- tip_amount: string (nullable = true)
 |-- tolls_amount: string (nullable = true)
 |-- ehail_fee: string (nullable = true)
 |-- improvement_surcharge: string (nullable = true)
 |-- total_amount: string (nullable = true)
 |-- payment_type: string (nullable = true)
 |-- trip_type: string (nullable = true)
 |-- congestion_surcharge: string (nullable = true)



In [14]:
from pyspark.sql import types

In [28]:
green_schema = types.StructType([
types.StructField('VendorID', types.IntegerType(), True), 
types.StructField('lpep_pickup_datetime', types.TimestampType(), True), 
types.StructField('lpep_dropoff_datetime', types.TimestampType(), True), 
types.StructField('store_and_fwd_flag', types.StringType(), True), 
types.StructField('RatecodeID', types.IntegerType(), True),
types.StructField('PULocationID', types.IntegerType(), True), 
types.StructField('DOLocationID', types.IntegerType(), True), 
types.StructField('passenger_count', types.IntegerType(), True), 
types.StructField('trip_distance', types.DoubleType(), True), 
types.StructField('fare_amount', types.DoubleType(), True), 
types.StructField('extra', types.DoubleType(), True), 
types.StructField('mta_tax', types.DoubleType(), True), 
types.StructField('tip_amount', types.DoubleType(), True), 
types.StructField('tolls_amount', types.DoubleType(), True), 
types.StructField('ehail_fee', types.DoubleType(), True), 
types.StructField('improvement_surcharge', types.DoubleType(), True), 
types.StructField('total_amount', types.DoubleType(), True), 
types.StructField('payment_type', types.IntegerType(), True), 
types.StructField('trip_type', types.IntegerType(), True), 
types.StructField('congestion_surcharge', types.DoubleType(), True)
])

In [29]:
df_green = spark.read \
    .option("header", "true") \
    .schema(green_schema) \
    .csv('data/raw/green/2021/01')

In [30]:
df_green.printSchema()

root
 |-- VendorID: integer (nullable = true)
 |-- lpep_pickup_datetime: timestamp (nullable = true)
 |-- lpep_dropoff_datetime: timestamp (nullable = true)
 |-- store_and_fwd_flag: string (nullable = true)
 |-- RatecodeID: integer (nullable = true)
 |-- PULocationID: integer (nullable = true)
 |-- DOLocationID: integer (nullable = true)
 |-- passenger_count: integer (nullable = true)
 |-- trip_distance: double (nullable = true)
 |-- fare_amount: double (nullable = true)
 |-- extra: double (nullable = true)
 |-- mta_tax: double (nullable = true)
 |-- tip_amount: double (nullable = true)
 |-- tolls_amount: double (nullable = true)
 |-- ehail_fee: double (nullable = true)
 |-- improvement_surcharge: double (nullable = true)
 |-- total_amount: double (nullable = true)
 |-- payment_type: integer (nullable = true)
 |-- trip_type: integer (nullable = true)
 |-- congestion_surcharge: double (nullable = true)



In [36]:
year = 2021

for month in range(1,13):
    print(f'processing data for {year}/{month}')

    input_path = f'data/raw/green/{year}/{month:02d}/'
    output_path = f'data/pq/green/{year}/{month:02d}'

    df_green = spark.read \
        .option("header", "true") \
        .schema(green_schema) \
        .csv(input_path)
    
    df_green \
        .repartition(4) \
        .write.parquet(output_path)

processing data for 2021/1
processing data for 2021/2
processing data for 2021/3
processing data for 2021/4
processing data for 2021/5
processing data for 2021/6
processing data for 2021/7
processing data for 2021/8


25/07/04 22:40:35 WARN FileStreamSink: Assume no metadata directory. Error while looking for metadata directory in the path: data/raw/green/2021/08/.
java.io.FileNotFoundException: File data/raw/green/2021/08 does not exist
	at org.apache.hadoop.fs.RawLocalFileSystem.deprecatedGetFileStatus(RawLocalFileSystem.java:917)
	at org.apache.hadoop.fs.RawLocalFileSystem.getFileLinkStatusInternal(RawLocalFileSystem.java:1238)
	at org.apache.hadoop.fs.RawLocalFileSystem.getFileStatus(RawLocalFileSystem.java:907)
	at org.apache.hadoop.fs.FilterFileSystem.getFileStatus(FilterFileSystem.java:462)
	at org.apache.spark.sql.execution.streaming.FileStreamSink$.hasMetadata(FileStreamSink.scala:56)
	at org.apache.spark.sql.execution.datasources.DataSource.resolveRelation(DataSource.scala:381)
	at org.apache.spark.sql.catalyst.analysis.ResolveDataSource.org$apache$spark$sql$catalyst$analysis$ResolveDataSource$$loadV1BatchSource(ResolveDataSource.scala:143)
	at org.apache.spark.sql.catalyst.analysis.Resolv

AnalysisException: [PATH_NOT_FOUND] Path does not exist: file:/Users/romadv-mac/dev_mac/romadv_dezoomcamp2025/5_batch_processing/code/data/raw/green/2021/08. SQLSTATE: 42K03

In [21]:
df_yellow_pd = pd.read_csv('data/raw/yellow/2021/01/yellow_tripdata_2021_01.csv.gz', nrows=1000)

In [23]:
spark.createDataFrame(df_yellow_pd).schema

StructType([StructField('VendorID', LongType(), True), StructField('tpep_pickup_datetime', StringType(), True), StructField('tpep_dropoff_datetime', StringType(), True), StructField('passenger_count', LongType(), True), StructField('trip_distance', DoubleType(), True), StructField('RatecodeID', LongType(), True), StructField('store_and_fwd_flag', StringType(), True), StructField('PULocationID', LongType(), True), StructField('DOLocationID', LongType(), True), StructField('payment_type', LongType(), True), StructField('fare_amount', DoubleType(), True), StructField('extra', DoubleType(), True), StructField('mta_tax', DoubleType(), True), StructField('tip_amount', DoubleType(), True), StructField('tolls_amount', DoubleType(), True), StructField('improvement_surcharge', DoubleType(), True), StructField('total_amount', DoubleType(), True), StructField('congestion_surcharge', DoubleType(), True)])

In [31]:
yellow_schema = types.StructType([
types.StructField('VendorID', types.IntegerType(), True), 
types.StructField('tpep_pickup_datetime', types.TimestampType(), True), 
types.StructField('tpep_dropoff_datetime', types.TimestampType(), True), 
types.StructField('passenger_count', types.IntegerType(), True), 
types.StructField('trip_distance', types.DoubleType(), True), 
types.StructField('RatecodeID', types.IntegerType(), True), 
types.StructField('store_and_fwd_flag', types.StringType(), True), 
types.StructField('PULocationID', types.IntegerType(), True), 
types.StructField('DOLocationID', types.IntegerType(), True), 
types.StructField('payment_type', types.IntegerType(), True), 
types.StructField('fare_amount', types.DoubleType(), True), 
types.StructField('extra', types.DoubleType(), True), 
types.StructField('mta_tax', types.DoubleType(), True), 
types.StructField('tip_amount', types.DoubleType(), True), 
types.StructField('tolls_amount', types.DoubleType(), True), 
types.StructField('improvement_surcharge', types.DoubleType(), True), 
types.StructField('total_amount', types.DoubleType(), True), 
types.StructField('congestion_surcharge', types.DoubleType(), True)
])

In [32]:
df_yellow = spark.read \
    .option("header", "true") \
    .schema(yellow_schema) \
    .csv('data/raw/yellow/2021/01')

In [33]:
df_yellow.printSchema()

root
 |-- VendorID: integer (nullable = true)
 |-- tpep_pickup_datetime: timestamp (nullable = true)
 |-- tpep_dropoff_datetime: timestamp (nullable = true)
 |-- passenger_count: integer (nullable = true)
 |-- trip_distance: double (nullable = true)
 |-- RatecodeID: integer (nullable = true)
 |-- store_and_fwd_flag: string (nullable = true)
 |-- PULocationID: integer (nullable = true)
 |-- DOLocationID: integer (nullable = true)
 |-- payment_type: integer (nullable = true)
 |-- fare_amount: double (nullable = true)
 |-- extra: double (nullable = true)
 |-- mta_tax: double (nullable = true)
 |-- tip_amount: double (nullable = true)
 |-- tolls_amount: double (nullable = true)
 |-- improvement_surcharge: double (nullable = true)
 |-- total_amount: double (nullable = true)
 |-- congestion_surcharge: double (nullable = true)



In [38]:
year = 2021

for month in range(1,13):
    print(f'processing data for yellow {year}/{month}')

    input_path = f'data/raw/yellow/{year}/{month:02d}/'
    output_path = f'data/pq/yellow/{year}/{month:02d}'

    df_yellow = spark.read \
        .option("header", "true") \
        .schema(yellow_schema) \
        .csv(input_path)
    
    df_yellow \
        .repartition(4) \
        .write.parquet(output_path)

processing data for yellow 2021/1


processing data for yellow 2021/2


processing data for yellow 2021/3


processing data for yellow 2021/4


processing data for yellow 2021/5


processing data for yellow 2021/6


processing data for yellow 2021/7


25/07/04 22:45:01 WARN FileStreamSink: Assume no metadata directory. Error while looking for metadata directory in the path: data/raw/yellow/2021/08/.
java.io.FileNotFoundException: File data/raw/yellow/2021/08 does not exist
	at org.apache.hadoop.fs.RawLocalFileSystem.deprecatedGetFileStatus(RawLocalFileSystem.java:917)
	at org.apache.hadoop.fs.RawLocalFileSystem.getFileLinkStatusInternal(RawLocalFileSystem.java:1238)
	at org.apache.hadoop.fs.RawLocalFileSystem.getFileStatus(RawLocalFileSystem.java:907)
	at org.apache.hadoop.fs.FilterFileSystem.getFileStatus(FilterFileSystem.java:462)
	at org.apache.spark.sql.execution.streaming.FileStreamSink$.hasMetadata(FileStreamSink.scala:56)
	at org.apache.spark.sql.execution.datasources.DataSource.resolveRelation(DataSource.scala:381)
	at org.apache.spark.sql.catalyst.analysis.ResolveDataSource.org$apache$spark$sql$catalyst$analysis$ResolveDataSource$$loadV1BatchSource(ResolveDataSource.scala:143)
	at org.apache.spark.sql.catalyst.analysis.Reso

processing data for yellow 2021/8


AnalysisException: [PATH_NOT_FOUND] Path does not exist: file:/Users/romadv-mac/dev_mac/romadv_dezoomcamp2025/5_batch_processing/code/data/raw/yellow/2021/08. SQLSTATE: 42K03